## Visualization of Shells

In [1]:
import itkwidgets as itkw
from vtkmodules.vtkIOXdmf2 import vtkXdmfReader
from vtkmodules.vtkCommonDataModel import vtkUnstructuredGrid
from vtkmodules.vtkFiltersGeometry import vtkGeometryFilter
from vtkmodules.vtkCommonExecutionModel import vtkStreamingDemandDrivenPipeline
import ipywidgets as ipw
import numpy as np
import pandas as pd

In [2]:
schedule = pd.read_csv('/home/amit/WorkSpace/UCLA/simulations/PhaseDiagram/Schedule.csv', sep='\t')

In [3]:
label = ipw.Label(value=r'\(\gamma = 0.2\), \(1/\beta = 0.01\)')
runw = ipw.IntSlider(min=0, max=2, step=1, value=0, description=r'Run', continuous_update=False)
filew = ipw.IntSlider(min=1, max=600, step=1, value=1, description=r'Simulation', continuous_update=False)
stepsw = ipw.IntSlider(min=0, max=999, step=1, value=0, description=r'Time Step', continuous_update=False)
box = ipw.VBox([label, ipw.HBox([runw, filew, stepsw])])

In [4]:
basedir = '/home/amit/WorkSpace/UCLA/simulations/PhaseDiagram/XDMFFiles'
initialfile = '{}/PhaseDiagram0/VTKFile-1.xdmf'.format(basedir)
rd = vtkXdmfReader()
rd.SetFileName(initialfile)
rd.UpdateInformation()
info = rd.GetExecutive().GetOutputInformation(0)
gf = vtkGeometryFilter()
gf.SetInputConnection(rd.GetOutputPort())
gf.Update()
pd = gf.GetOutput()
pd.GetPointData().SetActiveNormals('PointNormals')
vtkview = itkw.view(geometries=pd)

def selectfile(run, fid):
    gamma = schedule.iloc[fid - 1].Gamma
    temperature = schedule.iloc[fid - 1].Temperature
    label.value = r'\(\gamma = {0:9.3f}\), \(1/\beta = {1:6.4f}\)'.format(gamma, temperature)
    filename = '{}/PhaseDiagram{}/VTKFile-{}.xdmf'.format(basedir, run, fid)
    rd.SetFileName(filename)
    rd.UpdateInformation()
    info = rd.GetExecutive().GetOutputInformation(0)
    timesteps = info.Get(vtkStreamingDemandDrivenPipeline.TIME_STEPS())
    stepsw.max = timesteps[-1]
    stepsw.value = timesteps[-1]
    updateview(stepsw.value)
    
def updateview(time):
    info.Set(vtkStreamingDemandDrivenPipeline.UPDATE_TIME_STEP(), time)
    rd.Modified()
    gf.Update()
    pd.GetPointData().SetActiveNormals('PointNormals')
    vtkview.geometries = pd

In [5]:
filewidget = ipw.interactive_output(selectfile, {'run':runw, 'fid':filew})
viewwidget = ipw.interactive_output(updateview, {'time': stepsw} )
display(ipw.VBox([box, filewidget, viewwidget, vtkview]))